# Import Libraries

In [1]:
import cv2
import numpy as np
import math
import uuid

# Play video

In [2]:
# cap = cv2.VideoCapture('Data/drone_video_treino.mp4')
# print(cap.isOpened())

# while(cap.isOpened()):
#     ret, frame = cap.read()
    
#     resizedFrame = cv2.resize(frame, (960, 540))

#     try:
#         cv2.imshow("resizedFrame", resizedFrame)
#     except:
#         print("Can't show image!")
#         break

#     if cv2.waitKey(25) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()

# Detectors
Here we will use two different methods of detection.
- Haar Cascade: Is usually faster than most methods but do not have a great accuracy. It's important to notice that most images in the training set is images of the front and the side of the vehicles. So this is why the method performs poorly here.
- YOLO (You Only Look Once): Deep learning model trained on the COCO dataset. Robust model and preety accurate but is slower than haar cascade. This one performs the best.

## Haar cascade without tracker
- Poorly vehicles detection (detect stones)

In [3]:
# cap = cv2.VideoCapture('Data/Traffic_Example.mp4')

# car_cascade = cv2.CascadeClassifier('./detectors/car.xml')

# trackers = cv2.legacy.MultiTracker_create()

# count=0
# while(cap.isOpened()):
#     ret, frame = cap.read()

#     if (count % 50 == 0):
#         trackers = cv2.legacy.MultiTracker_create()
#         cars = car_cascade.detectMultiScale(frame)
#         for car in cars:
#             (x, y, w, h) = [int(v) for v in car]
#             cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)
            
#             # Track object
#             tracker = cv2.legacy.TrackerKCF_create()
#             trackers.add(tracker, frame, (x,y,w,h))
#     else:
#         success, boxes = trackers.update(frame)
#         for box in boxes:
#             (x, y, w, h) = [int(v) for v in box]
#             cv2.rectangle(frame, (x,y), (x+w, y+h), (255, 0, 0), 2)
#     count += 1

#     cars = car_cascade.detectMultiScale(frame)
#     for car in cars:
#         (x, y, w, h) = [int(v) for v in car]
#         cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)


#     # Display result ----------------------------------------------------------------------------
#     resizedFrame = cv2.resize(frame, (960, 540))
#     cv2.imshow("frame", resizedFrame)

#     if cv2.waitKey(25) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()

## YOLO detector without tracker

In [4]:
# Download yolo weights
import requests
url = 'https://drive.google.com/file/d/1arP01Q6KCFSRsqLbz8NilCKBU-ulj2dD/view?usp=sharing'
r = requests.get(url)
open('Data/yolov3.weights', 'wb').write(r.content)

with open("./Detectors/coco_classes.txt", 'r') as classes_file:
    CLASSES = dict(enumerate([line.strip() for line in classes_file.readlines()]))
with open("./Detectors/coco_classes_of_interest.txt", 'r') as coi_file:
    CLASSES_OF_INTEREST = tuple([line.strip() for line in coi_file.readlines()])
conf_threshold = 0.5
net = cv2.dnn.readNet("./Detectors/yolov3.weights", "./Detectors/yolov3.cfg")

def get_bounding_boxes(image):
    '''
    Return a list of bounding boxes of objects detected,
    their classes and the confidences of the detections made.
    '''

    # create image blob
    scale = 0.00392
    image_blob = cv2.dnn.blobFromImage(image, scale, (416, 416), (0, 0, 0), True, crop=False)

    # detect objects
    net.setInput(image_blob)
    layer_names = net.getLayerNames()
    output_layers = []
    for i in net.getUnconnectedOutLayers():
        output_layers.append(layer_names[i - 1])
    outputs = net.forward(output_layers)

    classes = []
    confidences = []
    boxes = []
    nms_threshold = 0.4

    for output in outputs:
        for detection in output:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold and CLASSES[class_id] in CLASSES_OF_INTEREST:
                width = image.shape[1]
                height = image.shape[0]
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                classes.append(CLASSES[class_id])
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])

    # remove overlapping bounding boxes
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)


    _bounding_boxes = []
    _classes = []
    _confidences = []
    for i in indices:
        _bounding_boxes.append(boxes[i])
        _classes.append(classes[i])
        _confidences.append(confidences[i])

    return _bounding_boxes, _classes, _confidences

# cap = cv2.VideoCapture('Data/Traffic_Example.mp4')
# print(cap.isOpened())

# while(cap.isOpened()):
#     ret, frame = cap.read()

#     _bounding_boxes, _classes, _confidences = get_bounding_boxes(frame)

#     for box in _bounding_boxes:
#         (x, y, w, h) = [int(v) for v in box]
#         cv2.rectangle(frame, (x,y), (x+w,y+h), (255,0,0), 2)

#     resizedFrame = cv2.resize(frame, (960, 540))
#     cv2.imshow("frame", resizedFrame)


#     if cv2.waitKey(25) & 0xFF == ord('q'):
#         break

# cap.release()
# cv2.destroyAllWindows()

# Tracker
- All tracker function used in the application
- Tracker utils functions

In [5]:
def add_new_blobs(boxes, classes, confidences, blobs, frame, mcdf):
    matched_blob_ids = []
    for i, box in enumerate(boxes):
        _type = classes[i] if classes is not None else None
        _confidence = confidences[i] if confidences is not None else None
        _tracker =  cv2.TrackerKCF_create()
        _tracker.init(frame, tuple(box))

        match_found = False
        for _id, blob in blobs.items():
            if get_overlap(box, blob.bounding_box) >= 0.6:
                match_found = True
                if _id not in matched_blob_ids:
                    blob.num_consecutive_detection_failures = 0
                    matched_blob_ids.append(_id)
                blob.update(box, _type, _confidence, _tracker)


        if not match_found:
            _blob = Blob(box, _type, _confidence, _tracker)
            blob_id = generate_object_id()
            blobs[blob_id] = _blob

    blobs = _remove_stray_blobs(blobs, matched_blob_ids, mcdf)
    return blobs

def remove_duplicates(blobs):
    for blob_id, blob_a in list(blobs.items()):
        for _, blob_b in list(blobs.items()):
            if blob_a == blob_b:
                break

            if get_overlap(blob_a.bounding_box, blob_b.bounding_box) >= 0.6 and blob_id in blobs:
                del blobs[blob_id]
    return blobs

def update_blob_tracker(blob, blob_id, frame):
    '''
    Update a blob's tracker object.
    '''
    success, box = blob.tracker.update(frame)
    if success:
        blob.num_consecutive_tracking_failures = 0
        blob.update(box)
    else:
        blob.num_consecutive_tracking_failures += 1

    return (blob_id, blob)

def _remove_stray_blobs(blobs, matched_blob_ids, mcdf):
    '''
    Remove blobs that "hang" after a tracked object has left the frame.
    '''
    for blob_id, blob in list(blobs.items()):
        if blob_id not in matched_blob_ids:
            blob.num_consecutive_detection_failures += 1
        if blob.num_consecutive_detection_failures > mcdf:
            del blobs[blob_id]
    return blobs

class Blob:
    '''
    A blob represents a tracked object as it moves around in a video.
    '''
    def __init__(self, _bounding_box, _type, _confidence, _tracker):
        self.bounding_box = _bounding_box
        self.type = _type
        self.type_confidence = _confidence
        self.centroid = get_centroid(_bounding_box)
        self.area = get_area(_bounding_box)
        self.tracker = _tracker
        self.num_consecutive_tracking_failures = 0
        self.num_consecutive_detection_failures = 0
        self.crossed_counting_line = False
        self.position_first_detected = tuple(self.centroid)

    def update(self, _bounding_box, _type=None, _confidence=None, _tracker=None):
        self.bounding_box = _bounding_box
        self.type = _type if _type is not None else self.type
        self.type_confidence = _confidence if _confidence is not None else self.type_confidence
        self.centroid = get_centroid(_bounding_box)
        self.area = get_area(_bounding_box)
        if _tracker:
            self.tracker = _tracker


'''
# Bounding box utility functions.
'''

def get_centroid(bbox):
    # Calculates the center point of a bounding box.
    x, y, w, h = bbox
    return (round((x + x + w) / 2), round((y + y + h) / 2))

def get_area(bbox):
    '''
    Calculates the area of a bounding box.
    '''
    _, _, w, h = bbox
    return w * h

def get_overlap(bbox1, bbox2):
    '''
    Calculates the degree of overlap of two bounding boxes.
    This can be any value from 0 to 1 where 0 means no overlap and 1 means complete overlap.
    The degree of overlap is the ratio of the area of overlap of two boxes and the area of the smaller box.
    '''
    bbox1_x1 = bbox1[0]
    bbox1_y1 = bbox1[1]
    bbox1_x2 = bbox1[0] + bbox1[2]
    bbox1_y2 = bbox1[1] + bbox1[3]

    bbox2_x1 = bbox2[0]
    bbox2_y1 = bbox2[1]
    bbox2_x2 = bbox2[0] + bbox2[2]
    bbox2_y2 = bbox2[1] + bbox2[3]

    overlap_x1 = max(bbox1_x1, bbox2_x1)
    overlap_y1 = max(bbox1_y1, bbox2_y1)
    overlap_x2 = min(bbox1_x2, bbox2_x2)
    overlap_y2 = min(bbox1_y2, bbox2_y2)

    overlap_width = overlap_x2 - overlap_x1
    overlap_height = overlap_y2 - overlap_y1

    if overlap_width < 0 or overlap_height < 0:
        return 0.0

    overlap_area = overlap_width * overlap_height

    bbox1_area = (bbox1_x2 - bbox1_x1) * (bbox1_y2 - bbox1_y1)
    bbox2_area = (bbox2_x2 - bbox2_x1) * (bbox2_y2 - bbox2_y1)
    smaller_area = bbox1_area if bbox1_area < bbox2_area else bbox2_area

    epsilon = 1e-5 # small value to prevent division by zero
    overlap = overlap_area / (smaller_area + epsilon)
    return overlap

def generate_object_id():
    return 'obj_' + uuid.uuid4().hex

# Counter
- Counter util functions

In [6]:
def _line_segments_intersect(line1, line2):
    def get_orientation(p, q, r):
        val = (q[1] - p[1]) * (r[0] - q[0]) - (q[0] - p[0]) * (r[1] - q[1])
        if val == 0:
            return 0
        return 1 if val > 0 else 2

    def is_on_segment(p, q, r):
        if q[0] <= max(p[0], r[0]) and q[0] >= min(p[0], r[0]) and \
            q[1] <= max(p[1], r[1]) and q[1] >= min(p[1], r[1]):
            return True
        return False

    p1 = line1[0]
    q1 = line1[1]
    p2 = line2[0]
    q2 = line2[1]

    o1 = get_orientation(p1, q1, p2)
    o2 = get_orientation(p1, q1, q2)
    o3 = get_orientation(p2, q2, p1)
    o4 = get_orientation(p2, q2, q1)

    if o1 != o2 and o3 != o4:
        return True

    if o1 == 0 and is_on_segment(p1, p2, q1):
        return True

    if o2 == 0 and is_on_segment(p1, q2, q1):
        return True

    if o3 == 0 and is_on_segment(p2, p1, q2):
        return True

    if o4 == 0 and is_on_segment(p2, q1, q2):
        return True

    return False

def _has_crossed_counting_line(bbox, line):
    '''
    Check if at least one edge of a bounding box is intersected by a counting line.
    '''
    x, y, w, h = bbox
    bbox_line1 = [(x, y), (x + w, y)]
    bbox_line2 = [(x + w, y), (x + w, y + h)]
    bbox_line3 = [(x, y), (x, y + h)]
    bbox_line4 = [(x, y + h), (x + w, y + h)]

    if _line_segments_intersect(bbox_line1, line) or \
            _line_segments_intersect(bbox_line2, line) or \
            _line_segments_intersect(bbox_line3, line) or \
            _line_segments_intersect(bbox_line4, line):
        return True
    return False


def attempt_count(blob, counting_lines, counts):
    for counting_line in counting_lines:
        if _has_crossed_counting_line(blob.bounding_box, counting_line) and not blob.crossed_counting_line:
            counts += 1
            blob.crossed_counting_line = True
            return blob, counts

    return blob, counts

# Visualizer
- Function used to draw bounding boxes and texts in the output frame

In [7]:
font = cv2.FONT_HERSHEY_DUPLEX
line_type = cv2.LINE_AA

def visualize(frame, blobs, fps, counts):
    # Display tracker and fps
    cv2.putText(frame, "KCF Tracker", (int(frame.shape[1]*0.05) ,int(frame.shape[0]*0.1)), font, 0.75, (50,170,50),2) 
    cv2.putText(frame, "FPS : " + str(int(fps)), (int(frame.shape[1]*0.05),int(frame.shape[0]*0.15)), font, 0.75, (50,170,50), 2)
    
    # draw and label blob bounding boxes
    for _id, blob in blobs.items():
        (x, y, w, h) = [int(v) for v in blob.bounding_box]
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        object_label = 'I: ' + _id[:8] \
            if blob.type is None \
            else 'I: {0}, T: {1} ({2})'.format(_id[:8], blob.type, str(blob.type_confidence)[:4])
        cv2.putText(frame, object_label, (x, y - 5), font, 1, (255, 0, 0), 2, line_type)

    # Draw counting line
    line_thickness = 2
    cv2.line(frame, (0, int(frame_shape[0]/2)), (frame_shape[1], int(frame_shape[0]/2)), (0, 255, 0), thickness=line_thickness)

    # Draw
    cv2.putText(frame, "Count: {}".format(counts), (int(frame.shape[1]*0.8), int(frame.shape[0]*0.1)), font, 1, (0, 0, 255), 2, line_type)
    
    
    return frame

# Detect car color

## KNN Classifier

In [8]:
import csv
import random
import math
import operator
import cv2


# calculation of euclidead distance
def calculateEuclideanDistance(variable1, variable2, length):
    distance = 0
    for x in range(length):
        distance += pow(variable1[x] - variable2[x], 2)
    return math.sqrt(distance)


# get k nearest neigbors
def kNearestNeighbors(training_feature_vector, testInstance, k):
    distances = []
    length = len(testInstance)
    for x in range(len(training_feature_vector)):
        dist = calculateEuclideanDistance(testInstance,
                training_feature_vector[x], length)
        distances.append((training_feature_vector[x], dist))
    distances.sort(key=operator.itemgetter(1))
    neighbors = []
    for x in range(k):
        neighbors.append(distances[x][0])
    return neighbors


# votes of neighbors
def responseOfNeighbors(neighbors):
    all_possible_neighbors = {}
    for x in range(len(neighbors)):
        response = neighbors[x][-1]
        if response in all_possible_neighbors:
            all_possible_neighbors[response] += 1
        else:
            all_possible_neighbors[response] = 1
    sortedVotes = sorted(all_possible_neighbors.items(),
                         key=operator.itemgetter(1), reverse=True)
    return sortedVotes[0][0]


# Load image feature data to training feature vectors and test feature vector
def loadDataset(
    filename,
    filename2,
    training_feature_vector=[],
    test_feature_vector=[],
    ):
    with open(filename) as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        for x in range(len(dataset)):
            for y in range(3):
                dataset[x][y] = float(dataset[x][y])
            training_feature_vector.append(dataset[x])

    with open(filename2) as csvfile:
        lines = csv.reader(csvfile)
        dataset = list(lines)
        for x in range(len(dataset)):
            for y in range(3):
                dataset[x][y] = float(dataset[x][y])
            test_feature_vector.append(dataset[x])


def knn_classifier(training_data, test_data):
    training_feature_vector = []  # training feature vector
    test_feature_vector = []  # test feature vector
    loadDataset(training_data, test_data, training_feature_vector, test_feature_vector)
    classifier_prediction = []  # predictions
    k = 3  # K value of k nearest neighbor
    for x in range(len(test_feature_vector)):
        neighbors = kNearestNeighbors(training_feature_vector, test_feature_vector[x], k)
        result = responseOfNeighbors(neighbors)
        classifier_prediction.append(result)
    return classifier_prediction[0]

## Feature Extraction

In [9]:
from PIL import Image
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import itemfreq

def color_histogram_of_test_image(test_src_image):

    # load the image
    image = test_src_image

    chans = cv2.split(image)
    colors = ('b', 'g', 'r')
    features = []
    feature_data = ''
    counter = 0
    for (chan, color) in zip(chans, colors):
        counter = counter + 1

        hist = cv2.calcHist([chan], [0], None, [256], [0, 256])
        # hist /= hist.sum() #normalize hist
                
        features.extend(hist)

        # find the peak pixel values for R, G, and B
        elem = np.argmax(hist)

        if counter == 1:
            blue = str(elem)
        elif counter == 2:
            green = str(elem)
        elif counter == 3:
            red = str(elem)
            feature_data = red + ',' + green + ',' + blue
            # print(feature_data)

    with open('test.data', 'w') as myfile:
        myfile.write(feature_data)


def color_histogram_of_training_image(img_name):

    # detect image color by using image file name to label training data
    if 'red' in img_name:
        data_source = 'red'
    elif 'yellow' in img_name:
        data_source = 'yellow'
    elif 'white' in img_name:
        data_source = 'white'
    elif 'black' in img_name:
        data_source = 'black'
    elif 'blue' in img_name:
        data_source = 'blue'
    elif 'violet' in img_name:
        data_source = 'violet'

    # load the image
    image = cv2.imread(img_name)

    chans = cv2.split(image)
    colors = ('b', 'g', 'r')
    features = []
    feature_data = ''
    counter = 0
    for (chan, color) in zip(chans, colors):
        counter = counter + 1

        hist = cv2.calcHist([chan], [0], None, [256], [0, 256])
        features.extend(hist)

        # find the peak pixel values for R, G, and B
        elem = np.argmax(hist)

        if counter == 1:
            blue = str(elem)
        elif counter == 2:
            green = str(elem)
        elif counter == 3:
            red = str(elem)
            feature_data = red + ',' + green + ',' + blue

    with open('training.data', 'a') as myfile:
        myfile.write(feature_data + ',' + data_source + '\n')


def training():

    # red color training images
    for f in os.listdir('./training_dataset/red'):
        color_histogram_of_training_image('./training_dataset/red/' + f)

    # yellow color training images
    for f in os.listdir('./training_dataset/yellow'):
        color_histogram_of_training_image('./training_dataset/yellow/' + f)

    # white color training images
    for f in os.listdir('./training_dataset/white'):
        color_histogram_of_training_image('./training_dataset/white/' + f)

    # black color training images
    for f in os.listdir('./training_dataset/black'):
        color_histogram_of_training_image('./training_dataset/black/' + f)

    # blue color training images
    for f in os.listdir('./training_dataset/blue'):
        color_histogram_of_training_image('./training_dataset/blue/' + f)

## Predict car color

In [10]:
def get_car_color_predictions(cars):
    predictions = [] # (car_id, predicted_label)

    for (car_id, car_img) in cars:
        color_histogram_of_test_image(car_img)
        prediction = knn_classifier('training.data', 'test.data')

        predictions.append((car_id, car_img, prediction))

    return predictions

# Utils

In [11]:
def get_car_counted(blobs_list, cars_counted):
    cars = [] # (car_id, car_img) 

    for blob in blobs_list:
        blob_id = blob[0]
        blob_obj = blob[1]
        
        if(blob_id not in cars_counted and blob_obj.crossed_counting_line):
            cars_counted.append(blob_id)

            x = int(blob_obj.bounding_box[0] - (blob_obj.bounding_box[0]+blob_obj.bounding_box[2])*0.04)
            y = int(blob_obj.bounding_box[1] - (blob_obj.bounding_box[1]+blob_obj.bounding_box[3])*0.08)
            w = int(blob_obj.bounding_box[0]+blob_obj.bounding_box[2] + (blob_obj.bounding_box[0]+blob_obj.bounding_box[2])*0.04)
            h = int(blob_obj.bounding_box[1]+blob_obj.bounding_box[3] + (blob_obj.bounding_box[1]+blob_obj.bounding_box[3])*0.08)
            car_img = frame[y:h, x:w]

            cars.append((blob_id, car_img))
            
    return cars

# Main Application

In [12]:
cap = cv2.VideoCapture('Data/Traffic_Example.mp4')
print(cap.isOpened())

car_cascade = cv2.CascadeClassifier('./car.xml')

cars_counted = []
cars_already_counted = 0

blobs = {}
max_consecutive_failures = 2
detection_interval = 5

ret, frame = cap.read()

frame = cv2.resize(frame, (960, 540))

frame_shape = frame.shape
print(frame_shape)
counting_lines = [
    [(0, int(frame_shape[0]/2)), (frame_shape[1], int(frame_shape[0]/2))]
]
counts = 0

output_video = cv2.VideoWriter("./Output/objectCounter.avi",
    cv2.VideoWriter_fourcc(*'MJPG'), 
    30,
    (frame.shape[1], frame.shape[0])
)

_bounding_boxes, _classes, _confidences = get_bounding_boxes(frame)
blobs = add_new_blobs(_bounding_boxes, _classes, _confidences, blobs, frame, max_consecutive_failures)

frame_count = 0
while(cap.isOpened()):

    frame = cv2.resize(frame, (960, 540))

    # Start timer
    timer = cv2.getTickCount()
    # Calculate Frames per second (FPS)
    fps = cv2.getTickFrequency() / (cv2.getTickCount() - timer)

    # update blob trackers
    blobs_list = list(blobs.items())
    blobs_list = [update_blob_tracker(blob, blob_id, frame) for blob_id, blob in blobs_list]
    blobs = dict(blobs_list)

    # Count vehicles
    for blob_id, blob in blobs_list:
        # count object if it has crossed a counting line
        blob, counts = attempt_count(blob, counting_lines, counts)
        blobs[blob_id] = blob
        # remove blob if it has reached the limit for tracking failures
        if blob.num_consecutive_tracking_failures >= max_consecutive_failures:
            del blobs[blob_id]

    # detect objects
    if frame_count % detection_interval == 0:
        # rerun detection
        _bounding_boxes, _classes, _confidences = get_bounding_boxes(frame)

        blobs = add_new_blobs(
            _bounding_boxes, _classes, _confidences, 
            blobs, frame, max_consecutive_failures)
        blobs = remove_duplicates(blobs)

    # Print blobs
    if(counts > cars_already_counted):
        cars_already_counted = counts

        cars = get_car_counted(blobs_list, cars_counted)

        car_color_predictions = get_car_color_predictions(cars)

        for (car_id, car_img, color_predicted) in car_color_predictions:
            cv2.imwrite('./CarsCounted/'+color_predicted+'_'+car_id+'.png', car_img)




        # for blob in blobs_list:
        #     if(blob[0] not in cars_counted and blob[1].crossed_counting_line):
        #         cars_counted.append(blob[0])

        #         x = int(blob[1].bounding_box[0] - (blob[1].bounding_box[0]+blob[1].bounding_box[2])*0.04)
        #         y = int(blob[1].bounding_box[1] - (blob[1].bounding_box[1]+blob[1].bounding_box[3])*0.08)
        #         w = int(blob[1].bounding_box[0]+blob[1].bounding_box[2] + (blob[1].bounding_box[0]+blob[1].bounding_box[2])*0.04)
        #         h = int(blob[1].bounding_box[1]+blob[1].bounding_box[3] + (blob[1].bounding_box[1]+blob[1].bounding_box[3])*0.08)
        #         carImg = frame[
        #             y:h,
        #             x:w
        #         ]
        #         # Prediction
        #         color_histogram_of_test_image(cropped_image)
        #         prediction = knn_classifier('training.data', 'test.data')
        #         cv2.imwrite('./CarsCounted/'+prediction+blob[0]+'.png', cropped_image)

    output_frame = visualize(frame, blobs, fps, counts)

    # Record frames
    output_video.write(output_frame)

    # Display result ----------------------------------------------------------------------------
    cv2.imshow("output_frame", output_frame)
    frame_count += 1
    ret, frame = cap.read()

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

True
(540, 960, 3)
